# Trump - a 140 character insight

---

## Initial research outline

As presidents lead their nation, we believe their actions could set an example and influence and encourage certain behaviors. We will investigate if there is a possible correlation between sentiment conveyed by such figures via social media and negative social behavior of a nation.

The presidency of Donald Trump has been marked with many controversies, including the rise of supremacist groups and numerous nation wide conflicts. By using Trump tweets, we would explore if there is a significant temporal correlation between sentiment expressed in the Tweet and the number of conflicts in the nation using GDELT dataset. To reduce the bias of such analysis, we would perform the same analysis with social media activity of his predecessor and compare the results, as well as adding the social-economic aspects which may affect such behaviors.

What is the power of a presidential tweet? We hope that we will have more insight on the answer of this question and raise awareness on the impact 140 characters can make.

### Research questions
* Do violent events or crimes occur more frequently after presidential tweets?
* Is the correlation higher with negative sentiment of the tweet?
* Does a significant difference exists between such correlation with Trump and Obama presidency?
* Is there a difference in sentiment expressed in tweets before and after Trump becoming a presidential candidate?
* How do social-economical aspects influence the correlation?
* Does the political regiment affect such negative behavior on a larger scale?

### Dataset

#### Trump tweets
* Downloading the complete dataset locally and processing the JSON files
* We would use mainly the timestamps and tweet content for our analysis:
  * timestamps for temporal correlation with events
  * tweet content for sentiment analysis
* We could follow the impact of tweet by analyzing:
  * number of retweets
  * number of favorites
  * number of followers
  
#### GDELT
* We would use GDELT Global Knowledge graph to gain insights on events:
  * geolocalized to USA
  * in specified timeframe, starting from 2013
  * specific events, such as protests or violent manifestations
  * will need knowledge on working on the cluster and accessing the data
  
#### Wikidata
* Obtaining data for social-economical indicators:
  * for a specific period of time
  * for a specific region of the USA
  * data wrangling and processing

#### Obama tweets *
Available at: http://obamawhitehouse.gov.archivesocial.com/

* It would enrich our analysis by providing an insight in differences between the two leaders
* We strive for a more complete and less biased analysis

---

## Evolution of research topic

The initial project proposal has evolved through iterations of data exploration, evaluation of project complexity and consultation with the teaching team in ADA.

### Main concerns that have been raised
* project required many areas of expertise
    * from social studies, behavioral studies, to mechanisms of USA economy
    * unable to reach the level of expertise on our own
    * for this type of project having external scientists would be an overkill
* timeframe for this project is limited
    * practically impossible to have a meaningful insight in about a month
* **signal detection and correlation would be very hard to prove since many underlying factors exist - complexity is high**
* GDELT could not provide the necessary granularity for this survey

A comment that described our situation and prospect of success:
> You will probably need a PhD to be able to come close to a meaningful result.

Even though we were passionate about our idea, we have realized we need to reduce the scope and pinpoint specific research questions that are feasible with our current expertise and available time. We are thankful to very insightful comments from professor Bob West who steered us to our current project. The project is reasonable, it is possibly insightful and we hope to squeeze as much information from the dataset as possible.

---

# TODO: current outline, abstract, research questions and datasets

# Data analysis

---

## General overview

We will perform an exploratory data analysis to get a deeper insight on the available data and information within. The two datasets we have decided on using are:

* Trump Twitter archive: *main dataset*
* Internet archive - Trump TV news factchecks: *enrichment dataset*


### Trump Twitter archive

Trump Twitter archive represents the complete collection of Tweets from the account of Donald Trump, since the first Tweet in 2009. The dataset has been obtained from [Trump Twitter Archive github repository](https://github.com/bpb27/trump_tweet_data_archive).

Internally, the dataset has been obtained by combining [Twitter scraper](https://github.com/bpb27/twitter_scraping) and official Twitter API. This combination would allow for scraping Tweets from any user account without a need to use the official premium Twitter API.

The result set is compliant with Twitter API output and information on meaning of columns in our dataset can be obtained by looking into [official documentation](https://developer.twitter.com/en/docs/tweets/tweet-updates).

More information and explorative data analysis will follow.

### Internet archive - Trump TV news factchecks

In further research we would like to have an insight how Donald Trump reacts to the news about him. Obtaining a dataset which collected the news about Donald Trump would be difficult because:
* Many sources online - which outlets to select
* Possible bias while covering significant portion of news
* Duplication of news from different sources
* Significant difference in time of publishing
* There is a need to extract main topics of news
* What is the criteria for marking the news are about Donald Trump - he will appear in many news stories!
* No universal API or method for this task - scraping of multiple sites would be necessary

Luckily, [The Internet Archive](https://archive.org/details/tv?factchecks) has a ready dataset combining televised news clips linked to Donald Trump, from 2009 until today. Most importantly for televised clips - the dates are present, as well as a short list of topics covered. This would make it easier to pinpoint the reaction, if there is any in the Tweets. 

More information about the dataset will follow in the exploratory data analysis.


### Where to process: in cluster or locally?
Both **Trump Twitter archive** and **Internet archive - Trump TV news factchecks** are small enough for a local, in-memory processing and analysis. Decisive characteristics of the datasets for a local, in-memory processing are:

* Trump Twitter archive:
    * multiple JSON files, split by the year of collection
    * content of the JSON files is specified by Twitter API
    * 9 uncondensed JSON files, in total less than 100MB (92MB)

* Internet archive - Trump TV news factchecks
    * single CSV file
    * less than 1 MB in size
    
---

## Exploratory data analysis

We will look in depth into the datasets and show the possible uses of the information, as well as comments on usability of certain parts of our data.

Since dataset size allows for working in-memory, we will use standard utilities such as *Pandas* and *Numpy*.

In [1]:
import pandas as pd
import numpy as np

In [2]:
TWEETS_PATH = 'C:/Users/Viktor/Desktop/trump_tweet_data_archive-master/trump_tweet_data_archive-master/master_' 
NEWS_PATH = 'data/factchecks.csv'

### Trump Twitter Archive

We combine all the present Tweets into a single `DataFrame` for an easier utilization. Since multiple dataframes are concatenated, we need to reset the index to obtain an unique index.

In [3]:
tweets = pd.DataFrame()

for i in range(2009,2018):
    tweets = tweets.append(pd.read_json(TWEETS_PATH+str(i)+'.json/master_'+str(i)+'.json'))
    
tweets.reset_index(inplace=True)

As well as observing all the present columns in our `DataFrame`:

In [5]:
tweets.columns

Index(['index', 'contributors', 'coordinates', 'created_at',
       'display_text_range', 'entities', 'extended_entities', 'favorite_count',
       'favorited', 'full_text', 'geo', 'id', 'id_str',
       'in_reply_to_screen_name', 'in_reply_to_status_id',
       'in_reply_to_status_id_str', 'in_reply_to_user_id',
       'in_reply_to_user_id_str', 'is_quote_status', 'lang', 'place',
       'possibly_sensitive', 'quoted_status', 'quoted_status_id',
       'quoted_status_id_str', 'retweet_count', 'retweeted',
       'retweeted_status', 'scopes', 'source', 'text', 'truncated', 'user',
       'withheld_copyright', 'withheld_in_countries', 'withheld_scope'],
      dtype='object')

Documentation provides more information about the semantics behind each attribute of a [Tweet object](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/tweet-object).

We are mainly interested in following general attributes of Tweets:
* **textual content**
* **number of retweets**
* **number of favorites**

Such Tweet context could be obtained from the fields:
* `text` - Tweet content
* `created_at` - timestamp of the tweet
* `entities` - *dictionary*, [entities](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/entities-object) parsed out of the tweet:
    * `hashtags` - list of included hashtags
    * `symbols` - list of symbols
    * `urls` - list of urls
    * `user_mentions` - list of mentioned users
* `favorite_count` - number of favorites
* `id` - Tweet id
* `is_quote_status` - indicates whether this is a quoted tweet - indicates that this is not a direct expression of the user
* `lang` - language of the tweet (could help in the NLP)
* `retweet_count` - number of retweets
* `source` - utility used to post the Tweet
* `user` - *dictionary*, information about the [user](https://developer.twitter.com/en/docs/tweets/data-dictionary/overview/user-object), we are interested in:
    * `followers count` - number of followers at the Tweet `created_at` time
    * `statuses_count` - number of tweets at the Tweet `created_at` time

---

**Sample Tweet:**

In [22]:
tweets.head(2)

,index,contributors,coordinates,created_at,display_text_range,entities,extended_entities,favorite_count,favorited,full_text,...,retweeted,retweeted_status,scopes,source,text,truncated,user,withheld_copyright,withheld_in_countries,withheld_scope
0,0,NaN,NaN,2009-12-23 17:38:18,NaN,"{'symbols': [], 'user_mentions': [], 'hashtags...",NaN,12,False,NaN,...,False,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",From Donald Trump: Wishing everyone a wonderfu...,False,"{'follow_request_sent': False, 'has_extended_p...",NaN,NaN,NaN
1,1,NaN,NaN,2009-12-03 19:39:09,NaN,"{'symbols': [], 'user_mentions': [], 'hashtags...",NaN,6,False,NaN,...,False,NaN,NaN,"<a href=""http://twitter.com"" rel=""nofollow"">Tw...",Trump International Tower in Chicago ranked 6t...,False,"{'follow_request_sent': False, 'has_extended_p...",NaN,NaN,NaN


**Sample `entities` dictionary**:

In [15]:
tweets.loc[0].entities

{'hashtags': [], 'symbols': [], 'urls': [], 'user_mentions': []}

**Sample `user` dictionary:**

In [16]:
tweets.loc[0].user

{'contributors_enabled': False,
 'created_at': 'Wed Mar 18 13:46:38 +0000 2009',
 'default_profile': False,
 'default_profile_image': False,
 'description': 'President-elect of the United States',
 'entities': {'description': {'urls': []},
  'url': {'urls': [{'display_url': 'DonaldJTrump.com',
     'expanded_url': 'http://www.DonaldJTrump.com',
     'indices': [0, 23],
     'url': 'https://t.co/mZB2hymxC9'}]}},
 'favourites_count': 45,
 'follow_request_sent': False,
 'followers_count': 15665560,
 'following': True,
 'friends_count': 41,
 'geo_enabled': True,
 'has_extended_profile': False,
 'id': 25073877,
 'id_str': '25073877',
 'is_translation_enabled': True,
 'is_translator': False,
 'lang': 'en',
 'listed_count': 45212,
 'location': 'New York, NY',
 'name': 'Donald J. Trump',
 'notifications': True,
 'profile_background_color': '6D5C18',
 'profile_background_image_url': 'http://pbs.twimg.com/profile_background_images/530021613/trump_scotland__43_of_70_cc.jpg',
 'profile_background_

We see many fields generated by the API, but as we have previously mentioned, we will use only a subset of such features for our analysis.

---

#### General characteristics of the dataset

In [32]:
print('Number of Tweets: ', tweets.shape[0])
print('Account created on: ', tweets.loc[0].user['created_at'])
print('User time zone:', tweets.loc[0].user['time_zone'])
print('Latest tweet time:', tweets.created_at.max())

Number of Tweets:  32532
Account created on:  Wed Mar 18 13:46:38 +0000 2009
User time zone: Eastern Time (US & Canada)
Latest tweet time: 2017-11-24 18:49:12


In [33]:
tweets.describe()

,index,contributors,favorite_count,id,id_str,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,possibly_sensitive,quoted_status_id,quoted_status_id_str,retweet_count,withheld_copyright
count,32532.000000,0.0,32532.000000,3.253200e+04,3.253200e+04,1.999000e+03,1.999000e+03,2.440000e+03,2.440000e+03,7489.0,2.860000e+02,2.860000e+02,32532.000000,1.0
mean,2965.614134,NaN,8414.287809,5.031478e+17,5.031478e+17,3.347165e+17,3.347165e+17,3.516381e+08,3.516381e+08,0.0,7.254326e+17,7.254326e+17,2572.550012,1.0
std,2112.973211,NaN,25144.002721,2.027952e+17,2.027952e+17,9.525507e+16,9.525507e+16,4.004455e+08,4.004455e+08,0.0,1.097156e+17,1.097156e+17,7422.315792,NaN
min,0.000000,NaN,0.000000,1.698309e+09,1.698309e+09,1.672774e+17,1.672774e+17,7.425000e+03,7.425000e+03,0.0,5.427798e+17,5.427798e+17,0.000000,1.0
25%,1193.000000,NaN,20.000000,3.350412e+17,3.350412e+17,2.937718e+17,2.937718e+17,4.251961e+07,4.251961e+07,0.0,6.293375e+17,6.293375e+17,16.000000,1.0
50%,2590.000000,NaN,68.000000,5.097998e+17,5.097998e+17,3.138054e+17,3.138054e+17,2.392729e+08,2.392729e+08,0.0,6.973068e+17,6.973068e+17,107.000000,1.0
75%,4443.000000,NaN,1910.000000,6.551359e+17,6.551359e+17,3.543118e+17,3.543118e+17,4.852951e+08,4.852951e+08,0.0,7.888950e+17,7.888950e+17,1151.250000,1.0
max,8143.000000,NaN,633253.000000,9.341318e+17,9.341318e+17,9.336626e+17,9.336626e+17,3.412873e+09,3.412873e+09,0.0,9.339594e+17,9.339594e+17,369530.000000,1.0


Donald Trump has had his Twitter account created on 18th of March, 2009. Since then he has Tweeted over 32532 times. We have taken a snippet of the dataset on 24th of November 2017 to work with locally. Latest tweet is posted on 18:49:12 that day, in Eastern time zone.

The dataset is being updated hourly with new tweets. After obtaining a stable tool for analysis and insight, it would be possible to perform an hourly update. Albeit, such miniscule changes in comparison to the total body of tweets would be insignificant for some analysis. 

As seen in the `DataFrame` description, across all the Tweets we can observe their overall coarse popularity:
* favorites: 
    * maximum: 633253
    * mean: 8414
    * median: 68
* retweets:
    * maximum: 369530 
    * mean: 2572
    * median: 107
    
We will look into greater detail about Tweet distribution over the years, their characteristics and stats in later phases of dedicated data analysis.

#### Geolocation - is there a value in it?

Twitter API provides geographical coordinates of the tweet, provided the user has enabled such an option. This could provide some insight on activities of any person using Twitter. In this case it could provide an option to see whether the tone of the Tweets and general topics differ when Donald Trump is in different locations: foreign countries, Oval office or resting at Mar-a-Lago.

As presumed, only certain number of Tweet is geotagged. This makes sense - since otherwise Secret Service might have a more difficult job in certain situations. Albeit, we can get a small glimpse of locations Donald Trump has visited, as we will demonstrate in a short example:

In [74]:
# we extract the tweets for which the geolocation is defined
coord = tweets.loc[tweets.coordinates.notnull()]['coordinates']
coord.shape

(2299,)

We will use `Folium` to show a map and place markers on locations Donald Trump has tweeted from.

In [47]:
import folium

m = folium.Map(location=[40.7624658, -73.9754123], zoom_start=2)

Since there are too many Tweets (2299) to visualize on a map, we presume that some have similar coordinates - we only want a general location. Thus we define functions to generalize the location and reduce the size of a list by merging similar coordinates together:

In [40]:
'''
Convenience function wrapper for reversing a list.

Parameters: 
my_list -- input list to reverse

Returns:
my_list -- list with reversed elements
'''
def my_reverse(my_list):
    my_list.reverse()
    return my_list

'''
Function for verifying whether a sublist exists in a list of lists.

Parameters:
my_list -- list of lists
check -- list, input for which to check existence in my_list

Returns:
boolean -- True if sublist does not exist in the list, False otherwise
'''
def not_in_list(my_list, check):
    for lst in my_list:
        if(np.array_equal(check, lst)):
            return False
    return True

'''
Function for removing similar coordinates from the coordinates iterable object. 
The similarity is defined by decimal precision - the higher the decimal it is more precise and less entries would be removed.

Parameters:
coord_list -- iterable object containing coordinates (pandas.Series object)
decimal -- the rounding precision to be applied

Returns:
np.array -- array with removed similar coordinates
'''
def remove_similar_coord(coord_list, decimal):
    cleaned = [[0.53242,  0.43315]]
    
    for el in coord_list:
        rounded = np.around(el['coordinates'],decimal)
        if (not_in_list(cleaned, rounded)):
            cleaned.append(rounded)
    
    return np.array(cleaned)

In [75]:
cleaned = remove_similar_coord(coord, 2)
cleaned.shape

(78, 2)

In comparison to 2299 markers, 78 markers would be more reasonable to plot on map. Only thing we did was generalizing Tweets that were posted from nearby locations. We will add the markers to the map, note that we need to reverse the coordinates provided in Twitter API (reverse the position of lattitude and longitude) to show the right location.

In [76]:
for el in cleaned:
    folium.Marker(location=np.flip(el, axis=0)).add_to(m)
    
m

Besides visualization, we can't reason much in terms of automated analysis from the map. One option would be to extract the location details from the coordinates, and then extract the required infomation (e.g. country name). This is possible by utilizing [`geopy` package](https://pypi.python.org/pypi/geopy), more specifically [`Nominatim`](http://wiki.openstreetmap.org/wiki/Nominatim), which represents a Python wrapper of the Nominatium API. We will showcase the example of extracting the country names from the previously visualized markers.

In [106]:
from geopy.geocoders import Nominatim

# we initialize the object - wrapper around Nominatim API
geolocator = Nominatim()

A short example on how to obtain the Location object and address:

In [107]:
# location is obtained by using the reverse method, and providing the coordinates
l = geolocator.reverse([40.7625069, -73.975321])

# address represents a field of the Location object, we see the country is last element separated by comma
l.address

'712 Fifth Avenue, 712, 5th Avenue, Diamond District, Manhattan Community Board 5, New York County, NYC, New York, 10019, United States of America'

In [114]:
cleaned.shape

(78, 2)

In [117]:
import time

countries_tweeted_from = []

for crd in cleaned:
    # some coordinates seem to be spoofed and are less than 1
    if(crd[0]>1 or crd[1]>1):
        # we flip the coordinate array and convert it to Python list
        coord_list = np.flip(crd, axis=0).tolist()
        
        # we get the Location object from coordinates
        location = geolocator.reverse(coord_list)
        
        # we split the address by comma character
        address = location.address.split(',')
        
        # the country represents the last element in the address list
        country = address[len(address)-1]
        
        # we append the extracted country to the list
        countries_tweeted_from.append(country)

We can now see the countries Donald Trump has Tweeted from:

In [118]:
set(countries_tweeted_from)

{' Hrvatska', ' India', ' Ireland', ' UK', ' United States of America', ' РФ'}

**Note:** for the current scope of our analysis and general project, this does not represent much value. It is here to showcase the possibility in certain use-cases.

In [ ]:
news = pd.read_csv(NEWS_PATH)